In [1]:
source('../../ABCA7lof2/prep_data.r')
library(tidyr)

In [5]:
sce

class: SingleCellExperiment 
dim: 19384 102710 
metadata(0):
assays(2): counts logcounts
rownames(19384): OR4F5 OR4F29 ... AC007325.4 AC007325.2
rowData names(2): X0 X1
colnames(102710): AAACCCAAGTAAACGT-1_0 AAACCCACAGAATTCC-1_1 ...
  TTTGGTTAGACACGAC-1_130528 TTTGGTTCAGGATCGA-1_130529
colData names(36): barcodes index ... x y
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

In [6]:
# load data
sce = readRDS('../../processed_data/sce.rds')
ex_layer_annos = read.csv('../../processed_data/Ex_layer_annotations.csv')
sce_subset = sce[,ex_layer_annos$barcodes]

In [7]:
print('summing...')
meta = sce_subset@colData
meta$layer = ex_layer_annos$layer
labels = as.data.frame(as.character(interaction(meta$layer, meta$projid)))
cell_labels = rownames(meta)
summed_counts_cellxind = sum_counts(assays(sce_subset)$counts, labels, cell_labels)


[1] "summing..."


In [8]:
# in how many cells per celltype is each gene detected?
counts_nonzero = assays(sce_subset)$counts>0
detected_genes_cell = sum_counts(counts_nonzero, label = as.data.frame(meta$layer), cell_labels)
fraction_detected_genes_cell = t(apply(detected_genes_cell$summed_counts, 1, function(x){x/detected_genes_cell$ncells}))
expressed10 = get_expressed_genes(fraction_detected_genes_cell, .10)

In [9]:
# get metadata summary
summary = read.csv('../../processed_data/single_cell_individual_metadata.csv', row.names='projid')
summary$APOE4 = ifelse(summary$apoe_genotype%in%c(24, 44, 34), 1, 0)
summary$LOF = summary$ABCA7LoF

In [10]:
# save all the data
all_data = list()

all_data[['summed_counts_by_ind']] = summed_counts_cellxind
all_data[['summary']] = summary
all_data[['expressed10']] = expressed10

saveRDS(all_data, '../../processed_data/ex_layer_stats_input.rds')